## Imports

In [46]:

from src import trees

from sklearn.datasets import load_iris
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [48]:
# Example with Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Create and train the classifier
clf = trees.DecisionTreeClassifier(max_depth=3)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print the tree structure
print("\nTree Structure:")
clf.print_tree()



(120, 4) (30, 4) (120,) (30,)
Accuracy: 1.00

Tree Structure:
If Feature[2] <= 1.95:
  Leaf: 0
Else:
  If Feature[2] <= 4.75:
    If Feature[3] <= 1.65:
      Leaf: 1
    Else:
      Leaf: 2
  Else:
    If Feature[3] <= 1.75:
      Leaf: 1
    Else:
      Leaf: 2


In [50]:
# Example with Wine dataset
wine = load_wine()
X = wine.data
y = wine.target
names = wine.feature_names


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Create and train the classifier
clf = trees.DecisionTreeClassifier(max_depth=5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Print the tree structure
print("\nTree Structure:")
clf.print_tree(feature_names=names)



(142, 13) (36, 13) (142,) (36,)
Accuracy: 0.94

Tree Structure:
If color_intensity <= 3.81:
  If proline <= 1002.50:
    If ash <= 2.93:
      Leaf: 1
    Else:
      Leaf: 0
  Else:
    Leaf: 0
Else:
  If flavanoids <= 1.40:
    Leaf: 2
  Else:
    If proline <= 724.50:
      If alcohol <= 13.14:
        Leaf: 1
      Else:
        Leaf: 0
    Else:
      Leaf: 0
